In [280]:
#Q1 Implement SVM from scratch

import numpy as np
import random
import matplotlib.pyplot as plt                                 #To plot scatter plot
from scipy.optimize import minimize, LinearConstraint           #For SVM optimization
from scipy.optimize import Bounds, BFGS                         #For SVM optimization

In [281]:
#1.1 Generate and visualize data
def gen_data():
    np.random.seed(105)
    Positive = np.concatenate((np.random.randn(10, 2) * 0.4 + [1.5, -0.5],
                             np.random.randn(10, 2) * 0.4 + [-1.5, 0.5]))
    Negative = np.random.randn(20, 2) * 0.3 + [0.0, -0.9]

    return Positive, Negative

In [282]:
#1.2 Plot data
def plot_data(Positive, Negative):
  x_axis_range = np.linspace(-2.5, 2.5, 20)
  plt.scatter(Positive[:, 0], Positive[:, 1], color = "b")
  plt.scatter(Negative[:, 0], Negative[:, 1], color = "r")

In [283]:
#1.3 Fit SVM with linear kernel with slack variable
def lagrange_dual(alpha, x, y, ZERO):
    result = 0
    non_zero_alphas = np.where(alpha > ZERO)[0]
    for i in non_zero_alphas:
        for k in non_zero_alphas:
            #result = result + alpha[i]*alpha[k]*y[i]*y[k]*np.dot(x[i, :], x[k, :]) 
            result = result + alpha[i]*alpha[k]*y[i]*y[k]*linear_kernel(x[i, :], x[k, :])
    result = 0.5*result - sum(alpha)     
    return result

In [284]:
def linear_kernel(x, y, **kwargs):
    # insert your code here 
    # return linear kernel output
    return np.dot(x, y)

In [285]:
def fit(X, Y, ZERO, C=None, kernel=None, kwargs = {}):
  
    '''
    X: input_data (Nx2) numpy array
    Y: binary target variable (N,) numpy array
    C: The slack variable
    kernel: kernel function (linear or polynomial)
    kwargs: dictionary of arguments
    '''
    # insert your code here
    # Apply optimization of dual function (Equation 7.10: Bishop)
    # make use of minimize()
    # 
    m, n = X.shape
    np.random.seed(1)
    
    alpha_0 = np.random.rand(m)*C                       # Initialize alphas to random values
    linear_constraint = LinearConstraint(Y, [0], [0])   # Define the constraint
    bounds_alpha = Bounds(np.zeros(m), np.full(m, C))   # Define the bounds

    # Find the optimal value of alpha
    result = minimize(lagrange_dual, alpha_0, args = (X, Y, ZERO), method='trust-constr', 
                      hess=BFGS(), constraints=[linear_constraint],
                      bounds=bounds_alpha)
    
    # The optimized value of alpha lies in result.x
    alpha = result.x
    return alpha


In [286]:
#Q1.4 Linear Kernel decision boundary visualization

def get_support_vectors(X, Y, lambdas, threshold=10e-5):
  # insert your code here
  # return support vectors, target variable values of the support vectors, lambdas of support vectoss
  # remeber only lambdas >=0 are to be kept
  s_vecs =  np.array([]); s_vecs_Y =  np.array([]); s_vecs_lambda =  np.array([])

  for x, y, lambda_m in zip (X, Y, lambdas):
    if lambda_m > threshold:
      s_vecs = np.append(s_vecs, x)
      s_vecs_Y = np.append(s_vecs_Y, y)
      s_vecs_lambda = np.append(s_vecs_lambda, lambda_m)

  return np.reshape(s_vecs, (-1, 2)), s_vecs_Y, s_vecs_lambda     #reshape a flattened s_vecs into shape = 10, 2, otherwise it's 20, 1

In [287]:
def predict(test_point, s_vecs, s_vecs_Y, s_vecs_lambda, C, ZERO, kernel, kwargs):
  '''
  test_point: test datapoint
  s_vecs: Support vectors 
  s_vecs_Y: target values of the Support vectors
  s_vec_lambda: Dual variables corresponding to the support vectors
  kernel, kwargs: kernel function (linear or polynomial)
  '''
  # insert your code here
  # Apply support vectors to predict the class of test_point (Equation 7.13: Bishop)

  #Calculate w1, w2
  number_of_samples = len(s_vecs)
  number_of_features = s_vecs.shape[1]

  weight = np.zeros(number_of_features)             #Number of features is the number of weights we want to find

  for i in range(number_of_samples):
    weight = weight + s_vecs_lambda[i] * s_vecs_Y[i] * s_vecs[i, :]

  #Calculate w0
  constraint = C - ZERO                            #because C cannot equal to lambda, thus we prepare to find lambda < C
  support_vendor_count = 0; w0 = 0

  for x, y, lambda_m in zip (s_vecs, s_vecs_Y, s_vecs_lambda):
    print("lambda_m", lambda_m, "constraint", constraint)
    if lambda_m > constraint:
      w0 = w0 + y - np.dot(x, weight)
      support_vendor_count += 1
  
  print("w0", w0)
  print("support_vendor_count", support_vendor_count)
  w0 = w0 / support_vendor_count

  prediction = np.sum(test_point * weight) + w0
  prediction = np.sign(prediction)              #np.sign returns -1, 0, 1, thus manually put 0 into 1 for labelling
  if prediction == 0:
    prediction = 1
  print("prediction", prediction)

  return prediction, w0, weight

In [288]:
def plot_decision_boundary(Positive, Negative,
                           ip_new,
                           s_vecs, s_vecs_Y, s_vec_lambda,
                           w0, weight,
                           kernel, kwargs):
  '''
  Positive, Negative: Example datapoints from both classes
  test_point: test data point (2, )
  s_vecs: Support vectors
  s_vecs_Y: Target variable values for the support vectors
  s_vec_lambda: Dual variables corresponding to the support vectors
  kernel, kwargs: kernel function (linear or polynomial)
  '''
  # make use of plt.contour() and plt.scatter()
  # plot decision boundary along with margins

  #plot hyperplane
  x1_coord = np.array(plt.gca().get_xlim())                           #can use linspace or whatever, the important point is to generate corresponding x of y
  x2_coord = np.array(plt.gca().get_ylim()) 
  x1_meshgrid, x2_meshgrid = np.meshgrid(x1_coord, x2_coord)
  y_coord = w0 + weight[0] *  x1_meshgrid + weight[1] * x2_meshgrid

  print("x1_meshgrid", x1_meshgrid)
  print("x2_meshgrid", x2_meshgrid)
  print("x1_coord", x1_coord)
  print("x2_coord", x2_coord)
  print("y", y_coord)
  plt.contour(x1_coord, x2_coord, y_coord, levels=[-1, 0, 1], colors=["blue", "black", "red"])

  #plot margin
  '''
  x2_positive_margin = 1/weight[1] -w0/weight[1] - weight[0]/weight[1] * x1_coord
  x2_negative_margin = -1/weight[1] -w0/weight[1] - weight[0]/weight[1] * x1_coord
  plt.plot(x1_coord, x2_positive_margin, color='blue')
  plt.plot(x1_coord, x2_negative_margin, color='red')

  #plot scatter
  x_axis_range = np.linspace(-2.5, 2.5, 20)
  plt.scatter(Positive[:, 0], Positive[:, 1], color = "b")
  plt.scatter(Negative[:, 0], Negative[:, 1], color = "r")
  '''
  
  

  return

In [ ]:
# Test code
if __name__ == '__main__':
    Positive, Negative = gen_data()
    plot_data(Positive, Negative)
    
    X = np.concatenate((Positive, Negative))
    target_data = np.concatenate((np.ones(Positive.shape[0]), -np.ones(Negative.shape[0])))

    size = X.shape[0]
    shuffle_data = list(range(size))      #range(40) means numbers  0 - 39 are created and lx12isted
    random.shuffle(shuffle_data)
    X = X[shuffle_data, :]
    Y = target_data[shuffle_data]
    ZERO = 1e-2
    C = 5

    lambdas = fit(X, Y, ZERO, C,  kernel=linear_kernel, kwargs={})    # C is slack variable
               
    # Get the alpha values for the support vectors, as well as their target data.
    s_vecs, s_vecs_Y, s_vecs_lambda = get_support_vectors(X, Y, lambdas)
    print(f"{s_vecs.shape[0]} support vectors found.\\n", s_vecs)

    print("s_vecs_Y", s_vecs_Y)
    print("s_vecs_lambda", s_vecs_lambda)
    
    # test new data point
    ip_new = np.array([-0.6, 0.75])
    prediciton, w0, weight = predict(ip_new, s_vecs, s_vecs_Y, s_vecs_lambda, C, ZERO, kernel=linear_kernel, kwargs={})
    
    plot_decision_boundary(  Positive, Negative,
                             ip_new,
                             s_vecs,
                             s_vecs_Y,
                             s_vecs_lambda,
                             w0, weight,
                             kernel=linear_kernel,
                             kwargs={})